In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init

In [3]:
# df = data.getInitData()
# data.mk_initData(df)
# first_np, second_np, label_np = utils.process(df)

In [4]:
first_np, second_np, label_np  = data.getData()
first_np, second_np = utils.process_splitted(first_np, second_np)
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

In [5]:
pretrained_word2vec = pm.load_pretrained_model(ch2idx)
pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 132 pretrained vectors found.


In [6]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [7]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=50)

In [8]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('ocr/tests')

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [9]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=len(ch2idx),
    embed_dim=200,
    hidden_size=100,
    num_classes=2,
    rnn_layers=1,
    dropout=0.0,
    learning_rate=0.001,
    model_name="RNN",
    optim_name="Adam",
    loss_fn_name="CEL",
    pretrained_model=pretrained_word2vec,
    freeze_embedding=False,
    device=device,
)

print(model)

doing with pretrained model!!!
OCR(
  (emb): Embedding(132, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)


In [10]:
trainer.train(
    epochs=20,
    title='test1',
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.693250   | 50.716049  | 0.693138 | 50.26  | 71.53 
   2    |   0.693257   | 50.057613  | 0.693178 | 50.26  | 71.41 
   3    |   0.693262   | 49.884774  | 0.693138 | 50.26  | 71.86 
   4    |   0.693214   | 49.806584  | 0.693175 | 49.74  | 72.44 
   5    |   0.693260   | 49.353909  | 0.693147 | 50.26  | 71.94 
   6    |   0.693201   | 49.152263  | 0.693178 | 49.74  | 70.55 
   7    |   0.693150   | 49.753086  | 0.693134 | 50.26  | 71.75 
   8    |   0.693169   | 49.769547  | 0.693150 | 49.74  | 71.70 
   9    |   0.693138   | 49.995885  | 0.693151 | 50.26  | 71.99 
  10    |   0.693176   | 49.987654  | 0.693185 | 49.74  | 71.89 
  11    |   0.693162   | 49.995885  | 0.693183 | 49.74  | 71.67 
  12    |   0.693143   | 50.176955  | 0.693298 | 49.74  | 71.23 
  13    |   0.693169   | 50.098765  | 0.693139 | 50.2